# Deploy mapping pipeline


# Install requirements

In [61]:
import sys
!{sys.executable} -m pip install python-dotenv

### Init workspace

In [63]:
from dotenv import dotenv_values
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import os

env_vars = dotenv_values("env")
for key, value in env_vars.items():
     os.environ[key] = value
subscription_id = os.getenv("SUBSCRIPTION_ID")
workspace_name = os.getenv("WORKSPACE_NAME")
version = os.getenv("VERSION")
print(f"workspace_name={workspace_name}")
print(f"version={version}")

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name="DocuBank",
    workspace_name=workspace_name,
)
cpu_cluster = None

workspace_name=test
version=0.0.4


### Create a compute resource to run your pipeline (Optional)


In [52]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster-lp"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure Machine Learning compute object with the intended parameters
    # if you run into an out of quota error, change the size to a comparable VM that is available.
    # Learn more on https://azure.microsoft.com/en-us/pricing/details/machine-learning/.
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure Machine Learning Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=600,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="LowPriority",
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

You already have a cluster named cpu-cluster-lp, we'll reuse it as is.


### Set Params

In [64]:
conda_file = "./conda.yaml"
code_dir = "./src"
env_name = "bankmap"
component_name = "bankmap"

config_path="azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/configs/paths/"
input_path="azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/input/paths/"

### Create environment


In [65]:
from azure.ai.ml.entities import Environment

pipeline_job_env = Environment(
    name=env_name,
    description="Env to run map pipeline",
    tags={"bankmap": version},
    conda_file=conda_file,
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version=version,
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name bankmap-tune is registered to workspace, the environment version is 0.0.4


### Create component


In [66]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

outputs={"output": Output(type="uri_file", mode="rw_mount")}
component = command(
    name=component_name,
    display_name="Tune bankmap params for a company",
    description="Tune limits for bankmap company",
    inputs={
        "data":  Input(type="uri_file"),
        "config_path":  Input(type="uri_folder"),
        "company": Input(type="string")
    },
    is_deterministic=True, 
    outputs=outputs,
    # The source folder of the component
    code=code_dir,
    version=version,
    command="""python map.py \
            --company '${{inputs.company}}' \
            --input_file '${{inputs.data}}'\
            --config_path '${{inputs.config_path}}'\
            --output '${{outputs.output}}'\
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

outputs {'output_path': {'type': 'uri_folder', 'path': 'azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/configs/paths/', 'mode': 'rw_mount'}}
outputs azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/configs/paths/


### Register the component in the workspace for future reuse.


In [67]:
# Now we register the component to the workspace
component = ml_client.create_or_update(component.component)

# Create (register) the component in your workspace
print(f"Component {component.name} with Version {component.version} is registered")

Uploading src (0.0 MBs): 100%|██████████| 3904/3904 [00:00<00:00, 47241.08it/s]




Component bankmap_tune with Version 0.0.4 is registered


### Optional: Test component



In [68]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output

tune_component = ml_client.components.get(component_name)
print(f'output: {tune_component}')

@dsl.pipeline(
    compute=cpu_compute_target,
    description="tune pipeline"
)
def tune_pipeline(company, data, config_path):
    job = tune_component(company=company, data=data, config_path=config_path)
    # job.outputs["output_path"] = Output(type="uri_folder", mode="rw_mount", path=output_path)
    print(f'output: {job.outputs["output"]}')
    return {
        "output": job.outputs.output
    }

output: name: bankmap_tune
version: 0.0.4
display_name: Tune bankmap params for a company
description: Tune limits for bankmap company
type: command
inputs:
  data:
    type: uri_file
    optional: false
  company:
    type: string
    optional: false
outputs:
  output_path:
    type: uri_folder
command: 'python tune.py             --company ''${{inputs.company}}''             --input_file
  ''${{inputs.data}}''            --output_path ''${{outputs.output_path}}''            '
environment: azureml:/subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourceGroups/DocuBank/providers/Microsoft.MachineLearningServices/workspaces/test/environments/bankmap-tune/versions/0.0.4
code: azureml:/subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourceGroups/DocuBank/providers/Microsoft.MachineLearningServices/workspaces/test/codes/4bf6f5c3-8ac8-4170-91de-08bd6b6d646e/versions/1
resources:
  instance_count: 1
creation_context:
  created_at: '2023-10-20T12:32:00.321476+00:00'
  created_by: "Ai

### Run job

In [69]:
# Let's instantiate the pipeline with the parameters of our choice
pipeline = tune_pipeline(
    company="hum",
    data=Input(type="uri_file", path="azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/devcopy/paths/hum.zip")
    config_path=Input(type="uri_folder", path=config_path)
)

pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    experiment_name="Map for test",
)
print(f'output before: {pipeline_job.outputs.output_path.path}')
# pipeline_job.outputs.output_path = Output(type="uri_folder", mode="rw_mount", path=output_path)
print(f'output: {pipeline_job.outputs.output.path}')
print(f'output: {pipeline_job.name}')

output: ${{parent.jobs.None.outputs.output_path}}
output before: azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/configs/paths/
output: azureml://subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test/datastores/configs/paths/
output: frosty_parrot_7gx54mp923


In [70]:
ml_client.jobs.stream(pipeline_job.name)

RunId: frosty_parrot_7gx54mp923
Web View: https://ml.azure.com/runs/frosty_parrot_7gx54mp923?wsid=/subscriptions/ae0eff97-7885-4c1e-b23c-d8a627ef292f/resourcegroups/DocuBank/workspaces/test

Streaming logs/azureml/executionlogs.txt

[2023-10-20 12:32:32Z] Submitting 1 runs, first five are: 6a6d3b43:d7099b8e-a71c-40da-957e-8702adc4fed7


In [91]:
output = ml_client.jobs.download(name=pipeline_job.name, download_path="./res", output_name="output")